## Reviewing Data of RPG Steam Games using Python and Pandas

Simple project to practice Pandas and explore Steam game data.

In [1]:
import numpy as np
import pandas as pd


# Obtaining Data

Data source is https://steamspy.com/ obtained 21 July 2021.

We used the SteamSpy API:
https://steamspy.com/api.php?request=genre&genre=RPG

I wanted to look at data from the RPG genre but this can be replaced with other genres.
This API provides a JSON file. I uploaded the file to my Google Drive so that it can be read in Colaboratory.

In [2]:
df = pd.read_json('drive/MyDrive/steamspyrpg.json')
df.head()

,252490,218620,238960,892970,49520,1091500,1468810,292030,291480,105600,301520,304050,413150,346110,219990,261550,251570,365590,72850,239140,588430,377160,582660,363970,285800,109600,555570,489830,435150,204360,356190,306130,489520,280790,8980,1145360,211820,39210,42910,1286830,...,1541400,1513400,1534590,1531800,1540740,1542010,1532350,1549650,1484660,1549240,1533650,1546280,1554760,1555230,1483300,1553910,1543550,1397300,1543590,1192630,1542820,1480870,226920,749820,681740,716780,670970,405680,704520,763000,698700,554710,801080,599490,593150,652730,611820,701360,643930,721460
appid,252490,218620,238960,892970,49520,1091500,1468810,292030,291480,105600,301520,304050,413150,346110,219990,261550,251570,365590,72850,239140,588430,377160,582660,363970,285800,109600,555570,489830,435150,204360,356190,306130,489520,280790,8980,1145360,211820,39210,42910,1286830,...,1541400,1513400,1534590,1531800,1540740,1542010,1532350,1549650,1484660,1549240,1533650,1546280,1554760,1555230,1483300,1553910,1543550,1397300,1543590,1192630,1542820,1480870,226920,749820,681740,716780,670970,405680,704520,763000,698700,554710,801080,599490,593150,652730,611820,701360,643930,721460
name,Rust,PAYDAY 2,Path of Exile,Valheim,Borderlands 2,Cyberpunk 2077,鬼谷八荒,The Witcher 3: Wild Hunt,Warface,Terraria,Robocraft,Trove,Stardew Valley,ARK: Survival Evolved,Grim Dawn,Mount & Blade II: Bannerlord,7 Days to Die,Tom Clancy’s The Division,The Elder Scrolls V: Skyrim,Dying Light,Fallout Shelter,Fallout 4,Black Desert,Clicker Heroes,Braveland,Neverwinter,Infestation: The New Z,The Elder Scrolls V: Skyrim Special Edition,Divinity: Original Sin 2 - Definitive Edition,Castle Crashers,Middle-earth: Shadow of War,The Elder Scrolls Online,Minion Masters,Creativerse,Borderlands Game of the Year,Hades,Starbound,FINAL FANTASY XIV Online,Magicka,STAR WARS: The Old Republic,...,Cutthroat Cove,Karma - Chapter 1,Farm Dungeons,Retro Racer,Dungen,Mariuccha Alchemy Queen,Rogue Explorer,The Last Place,Drabidiboo,扫雷冒险谭2 ~露露姆的冒险~,Knight Throde,Mad Pack,Blessings of No-more,Starvoly,MazM: The Phantom of the Opera,My Friend Stalin,Eviron's Chronicles,Kyma Maker's Island,Wildshape - Map Editor + VTT,23UL,Roguely,Lightning Fast,Hellraid,Villages,Liberator TD,Sand is the Soul,Res Judicata: Vale of Myth,Sentou Gakuen: Revival,Battle Buddies VR,Super Army of Tentacles 3: The Search for Army...,Rise of the Third Power,Don't Die: Survival,Hunting on Myths,Bounty Killer,Ooblets,Z-Aftershock,Xenosis: Alien Infection,Fate Crawler,拯救大魔王3 Falsemen3,Xeno Time Inception
developer,Facepunch Studios,OVERKILL - a Starbreeze Studio.,Grinding Gear Games,Iron Gate AB,"Gearbox Software, Aspyr (Mac), Aspyr (Linux)",CD PROJEKT RED,鬼谷工作室,CD PROJEKT RED,MY.GAMES,Re-Logic,Freejam,Trion Worlds,ConcernedApe,"Studio Wildcard, Instinct Games, Efecto Studio...",Crate Entertainment,TaleWorlds Entertainment,The Fun Pimps,Massive Entertainment,Bethesda Game Studios,Techland,Bethesda Game Studios,Bethesda Game Studios,Pearl Abyss,Playsaurus,Tortuga Team,Cryptic Studios,Fredaikis AB,Bethesda Game Studios,Larian Studios,The Behemoth,Monolith Productions,Zenimax Online Studios,BetaDwarf,Playful Corp.,Gearbox Software,Supergiant Games,Chucklefish,Square Enix,Arrowhead Game Studios,BioWare,...,Wicked Games,The Exact Co.,JRPG.Novels,Sweet Games,SVGames,Nuskadamo,Zoo Corporation,Lghghre GAME,Morgan Vitale,CelLab,LIU Daohuan,Liga Robotov,JJBenefield Games,Zeratd Gaming,Growing Seeds,Sweet Games,Time Stealers Studio,Kidness,DLDev,FLOR3,SQRL Studios,Steppe Games,Techland,HAILFALLKOMPANY,Okay Studios,MGP Studios,Blue Booth Studios,PST Team,C5A,Stegalosaurus Game Development,Stegosoft Games,dontdiedevelop,Birnchen Studios,Galaxy Game Studio,Glumberland,Mango Tree Game,NerdRage Studios,MixerGames,吃了就睡工作室,Xeno Gaming LLC
publisher,Facepunch Studios,Starbreeze Publishing AB,Grinding Gear Games,Coffee Stain Publishing,"2K, Aspyr (Mac), Aspyr (Linux)",CD PROJEKT RED,"鬼谷工作室, Lightning Games",CD PROJEKT RED,MY.GAMES,Re-Logic,Freejam,Trion W

Pandas created a dataframe with the app ids as columns so we use transpose to fix it.

In [3]:
df = df.transpose()
df.head()


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
252490,252490,Rust,Facepunch Studios,Facepunch Studios,,562708,87928,0,"20,000,000 .. 50,000,000",15560,1127,2645,463,3999,3999,0,113970
218620,218620,PAYDAY 2,OVERKILL - a Starbreeze Studio.,Starbreeze Publishing AB,,464746,60729,0,"10,000,000 .. 20,000,000",5420,1189,676,268,999,999,0,32322
238960,238960,Path of Exile,Grinding Gear Games,Grinding Gear Games,,152443,12073,0,"10,000,000 .. 20,000,000",8371,728,287,643,0,0,0,10500
892970,892970,Valheim,Iron Gate AB,Coffee Stain Publishing,,242821,10213,0,"10,000,000 .. 20,000,000",2396,556,1459,337,1999,1999,0,26812
49520,49520,Borderlands 2,"Gearbox Software, Aspyr (Mac), Aspyr (Linux)","2K, Aspyr (Mac), Aspyr (Linux)",,232676,15992,0,"10,000,000 .. 20,000,000",3487,206,1017,197,1999,1999,0,3849


# Working on Data

I chose to focus on the user scores. Created a new dataframe with the app id, name, and positive and negative scores.

In [4]:
df2 = df.loc[df['positive']>0, ['appid','name','positive','negative']].copy()
df2.head()

,appid,name,positive,negative
252490,252490,Rust,562708,87928
218620,218620,PAYDAY 2,464746,60729
238960,238960,Path of Exile,152443,12073
892970,892970,Valheim,242821,10213
49520,49520,Borderlands 2,232676,15992


Created a new column to calculate positive userscore %.

In [5]:
df2['score'] = df2['positive']/(df2['positive']+df2['negative']) * 100
df2.head()

,appid,name,positive,negative,score
252490,252490,Rust,562708,87928,86.4858
218620,218620,PAYDAY 2,464746,60729,88.443
238960,238960,Path of Exile,152443,12073,92.6615
892970,892970,Valheim,242821,10213,95.9638
49520,49520,Borderlands 2,232676,15992,93.5689


Checked if there are nulls - none found.

In [6]:
df2.isnull().sum()

appid       0
name        0
positive    0
negative    0
score       0
dtype: int64

Checked if there are game duplicates - found a few.

In [7]:
df2.duplicated(subset='name').sum()

6

In [8]:
df2[df2.duplicated(subset='name')]

,appid,name,positive,negative,score
22490,22490,Fallout: New Vegas,46753,2153,95.5977
1140510,1140510,Fantasy Gladiators,8,2,80
1526180,1526180,Hide and Seek,37,13,74
570850,570850,Metal Assault,9,4,69.2308
1382380,1382380,Aurora,4,0,100
1456750,1456750,Ceres,10,19,34.4828


Upon checking the data based on game name, we found that these games are not duplicates and have different app ids. It's appears that the duplicates are a different version of the same game, or a different game with the same name.

In [9]:
df2[df2['name'].str.contains(('Fallout: New Vegas'), case=False)]


,appid,name,positive,negative,score
22380,22380,Fallout: New Vegas,128391,4790,96.4034
22490,22490,Fallout: New Vegas,46753,2153,95.5977


Sorted data to look at the highest scoring games.

In [10]:
df2.sort_values(by='score', ascending=False)

,appid,name,positive,negative,score
1283240,1283240,Center2048,1,0,100
1264350,1264350,Echoes of Nocturnal Chords 夜弦之音,46,0,100
1539250,1539250,Karakuri-kun: A Japanese Tale,3,0,100
1212640,1212640,A Lanterns Glow,2,0,100
861580,861580,Delve,2,0,100
...,...,...,...,...,...
321390,321390,Zone 22,1,11,8.33333
1509640,1509640,Viking Warrior,1,12,7.69231
479180,479180,Space Universe,2,26,7.14286
917600,917600,Became The Hunted,1,13,7.14286


We found multiple games with 100% score but very few votes so we filter to those with more than 10000 scores.

In [11]:
df2[df2['positive'] > 10000].sort_values(by='score', ascending=False)

,appid,name,positive,negative,score
1145360,1145360,Hades,150146,1898,98.7517
1150690,1150690,OMORI,20479,282,98.6417
292030,292030,The Witcher 3: Wild Hunt,519405,10802,97.9627
413150,413150,Stardew Valley,365554,7757,97.9221
105600,105600,Terraria,761081,16217,97.9137
...,...,...,...,...,...
424370,424370,Wolcen: Lords of Mayhem,33947,27032,55.67
355840,355840,Survarium,13169,11136,54.1823
555570,555570,Infestation: The New Z,17057,15350,52.6337
834910,834910,ATLAS,17980,20602,46.602
